In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import log_loss

# Загрузка данных из файла insclass_train.csv
data = pd.read_csv('insclass_train.csv')

# Выделение признаков (features) и целевой переменной (target)
X = data.drop(columns=['target'])
y = data['target']

# Применение Label Encoding к категориальным признакам
labelEncoder = LabelEncoder()
categorical_columns = ['variable_1', 'variable_4', 'variable_5', 'variable_18', 'variable_20', 'variable_21', 'variable_22', 'variable_23', 'variable_28']
for col in categorical_columns:
    X[col] = labelEncoder.fit_transform(X[col])


# Создаем тестовый датасет
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=22624, random_state=42)

class LogisticRegression(object):
    def __init__(self, n):
        self.a = np.zeros(n).reshape(1,n)

    def sigmoid(self, X):
        return 1/(1 + np.exp(-X @ self.a))

    def predict(self, X):
        return self.sigmoid(X)

    def coefs(self):
        return self.a

    def LogLikelihood(self, X, y):
        p = self.predict(X)
        return (y * np.log(p) + (1 - y) * np.log(1 - p)).sum()

    def CrossEntropy(self, X, y):
        p = self.predict(X)
        return (-y * np.log(p) - (1 - y) * np.log(1 - p)).sum()

    def accuracy(self, X, y):
        return ((self.predict(X) > 0.5) == y).mean()

    def fit(self, X, y, alpha=0.001, epsylon=0.01, max_steps=5000, method="LL"):
        self.a = np.zeros(X.shape[1]).reshape(X.shape[1], 1)
        steps, errors = [], []
        step = 0

        for _ in range(max_steps):
            if method == "LL":
                new_error = self.LogLikelihood(X, y)
                dT_a = X.T @ (y - self.predict(X))
                self.a += alpha * dT_a
            elif method == "CE":
                new_error = self.CrossEntropy(X, y)
                dT_a = -X.T @ (y - self.predict(X))
                self.a -= alpha * dT_a

            step += 1
            steps.append(step)
            errors.append(new_error)

        return steps, errors


# Выбор модели
model_LL = LogisticRegression(n=X_test.shape[1])
model_CE = LogisticRegression(n=X_test.shape[1])

# Обучение модели на обучающем наборе
steps_LL, errors_LL = model_LL.fit(X, y, alpha=0.01, max_steps=2000, method="LL")
steps_CE, errors_CE = model_CE.fit(X, y, alpha=0.01, max_steps=2000, method="CE")

# Предсказание на тестовом наборе
y_pred_LL = model_LL.predict(X_test)
y_pred_CE = model_CE.predict(X_test)

# Сохранение предсказаний в файл
predicted_LL_data = pd.DataFrame({'target': y_pred_LL})
predicted_LL_data.to_csv('predicted_LL_test_data.csv', index=False)

predicted_CE_data = pd.DataFrame({'target': y_pred_CE})
predicted_CE_data.to_csv('predicted_CE_test_data.csv', index=False)


MemoryError: Unable to allocate 171. GiB for an array with shape (151405, 151406) and data type float64